In [2]:
# Import Libraries

import pandas as pd
import os
import requests
import psycopg2
import psycopg2.extras
import json
import datanews
import time

In [3]:
# Datanews API Credentials

api_key = "0fdlbukyla9uenffppsig1faq"


In [4]:
# Dates we will collect news for the general market

from_dates = ["2020-04-01", "2020-05-01","2020-06-01","2020-07-01","2020-08-01","2020-09-01","2020-10-01",
             "2020-11-01","2020-12-01","2021-01-01","2021-02-01","2021-03-01"]

to_dates = ["2020-05-01","2020-06-01","2020-07-01","2020-08-01","2020-09-01","2020-10-01","2020-11-01",
            "2020-12-01","2021-01-01","2021-02-01","2021-03-01", "2021-04-01"]


In [5]:
# Retreive news for each stock for the past year in increments of months

news_requests = []

for i in range(len(to_dates)):
    params = {'apiKey': api_key,
              'q': "stock market",
              'from': from_dates[i],
              'to': to_dates[i],
              'language': 'en',
              'size': 100}

    r = requests.get("http://api.datanews.io/v1/news", params=params)

    news_requests.append(r)

    time.sleep(3)



In [6]:
# Store news in a dataframe

news_data = pd.DataFrame()

for i in news_requests:
    
    if news_data.empty == True:
        news_data = pd.DataFrame(i.json()['hits'])
    else:
        try:
            mn = pd.DataFrame(i.json()['hits'])
        except:
            pass
        
        news_data = news_data.append(mn)

In [7]:
news_data.shape

(1200, 10)

In [8]:
# remove any rows with null values
news_data = news_data.dropna(how='any',axis=0) 
news_data.shape

(1168, 10)

In [9]:
news_data.head()

,url,source,authors,title,pubDate,country,language,description,imageUrl,content
0,https://www.ccn.com/this-coronavirus-insurance...,ccn.com,[],This Coronavirus Insurance Bill Could Crash th...,2020-04-06T13:26:55+00:00,us,en,Congress is examining a coronavirus insurance ...,https://www.ccn.com/wp-content/uploads/2020/04...,\n• Congress is considering a Pandemic Risk In...
1,https://www.ccn.com/chinas-scary-coronavirus-w...,ccn.com,[],China's Scary Coronavirus Warning Risks Epic S...,2020-04-28T07:40:02+00:00,us,en,The U.S. stock market saw a 14 percent recover...,https://www.ccn.com/wp-content/uploads/2020/04...,\n• Top scientists in China claim coronavirus ...
2,http://www.smarteranalyst.com/analyst-insights...,smarteranalyst.com,"[Ben Mahaney, Editor]",Is Coca-Cola Stock the Real Deal?,2020-04-15T06:15:01+00:00,,en,"Like most stocks in this market, Coca-Cola Com...",https://www.smarteranalyst.com/wp-content/uplo...,"Like most stocks in this market, Coca-Cola Com..."
3,https://marketrealist.com/2020/04/is-axp-stock...,marketrealist.com,[Ambrish Shah],Is AXP Stock a ‘Sell’ after Its Q1 Earnings?,2020-04-27T12:01:16+00:00,,en,"Today at 7:02 AM ET, AXP stock rose 1.1% to $8...",https://media.marketrealist.com/brand-img/FGgX...,American Express stock rose 0.86% on April 24 ...
4,https://www.ajc.com/news/state--regional-govt-...,ajc.com,"[Tia Mitchell, Chris Joyner]","Loeffler reports more stock sales, denies wron...",2020-04-06T21:27:18+00:00,us,en,"Loeffler reports more stock sales, denies wron...",https://www.ajc.com/resizer/nuSPA3uLGiA84c4fOT...,Loeffler and her staff have said she did nothi...


In [10]:
# Replace all comas with spaces, since exporting to csv was generating wrong with comas included
news_data['url'] = news_data['url'].replace(',','', regex=True)
news_data['source'] = news_data['source'].replace(',','', regex=True)
news_data['title'] = news_data['title'].replace(',','', regex=True)


In [11]:
# Store market news data in PostgreSQL database

conn = None

try:
    conn = psycopg2.connect(host="localhost", database="postgres", port=5432, user="postgres", password="Aki12345+")

    command = "INSERT INTO market_news (headline, source, url, date) \
    VALUES (%s, %s, %s, %s);"

    cur = conn.cursor()
    for index, row in news_data.iterrows():
        cur.execute(command, (str(row['title']),str(row['source']),
                              str(row['url']), str(row['pubDate'])))

except (Exception, psycopg2.DatabaseError):
    print("Could not connect to database")

finally:
    if conn is not None:
        conn.commit()
        conn.close()
        cur.close()